# Add a new plugin

By default, the library will import all the files with prefix "plugin\_\*.py" from src/synthcity/plugins, and load all the classes which implement the [Plugin interface](src/synthcity/plugins/core/plugin.py).

Each plugin must implement the following methods:
- hyperparameter_space() - a static method that returns the hyperparameters that can be tuned during AutoML.
- type() - a static method that returns the type of the plugin. e.g., debug, generative, bayesian, etc.
- name() - a static method that returns the name of the plugin. e.g., ctgan, random_noisee, etc.
- _fit() - internal method, called by `fit` on each training set.
- _generate() - internal method, called by `generate`.

## Existing plugins

In [1]:
from synthcity.plugins import Plugins

generators = Plugins()

generators.list()

['rtvae',
 'copulagan',
 'pategan',
 'tvae',
 'gaussian_copula',
 'adsgan',
 'marginal_distributions',
 'bayesian_network',
 'uniform_sampler',
 'privbayes',
 'nflow',
 'ctgan']

## Example plugin: Generate 0-1

In [2]:
# stdlib
from typing import Any, List

# third party
import pandas as pd
import numpy as np

# synthcity absolute
from synthcity.plugins.core.distribution import Distribution
from synthcity.plugins.core.plugin import Plugin
from synthcity.plugins.core.schema import Schema


class ZeroOnePlugin(Plugin):
    """Dummy plugin for debugging.
    """

    def __init__(self, **kwargs: Any) -> None:
        super().__init__(**kwargs)

    @staticmethod
    def name() -> str:
        return "zero_one"

    @staticmethod
    def type() -> str:
        return "debug"

    @staticmethod
    def hyperparameter_space(*args: Any, **kwargs: Any) -> List[Distribution]:
        return []

    def _fit(self, X: pd.DataFrame, *args: Any, **kwargs: Any) -> "ZeroOnePlugin":
        self.features_count = X.shape[1]
        return self

    def _generate(self, count: int, syn_schema: Schema, **kwargs: Any):
        return np.random.randint(0, 2, size=(count, self.features_count))

In [3]:
# Add the new plugin to the collection

generators.add("zero_one", ZeroOnePlugin)

In [4]:
# Check the new plugins list
generators.list()

['rtvae',
 'copulagan',
 'pategan',
 'tvae',
 'gaussian_copula',
 'adsgan',
 'marginal_distributions',
 'bayesian_network',
 'uniform_sampler',
 'privbayes',
 'nflow',
 'ctgan']

In [5]:
# Load reference data

from sklearn.datasets import load_breast_cancer

X, y = load_breast_cancer(return_X_y = True, as_frame = True)

X

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


In [6]:
# Train the new plugin

gen = generators.get("zero_one")

gen.fit(X)

In [7]:
# Generate some new data

gen.generate(count = 10)

RuntimeError: Plugin zero_one failed to meet the synthetic constraints.

### Oops, this didn't work.

__The Plugin interface enforces the new generated data to:__
 - satistify the same constraints as the training set.
 - Or to satisfy the constraints provided at inference time(if provided).
 
 
 If the generated dataframe fails to comply, an exception will be raised.

Let's try again

## A functional plugin

In [8]:
# stdlib
from typing import Any, List

# third party
import pandas as pd
import numpy as np

# synthcity absolute
from synthcity.plugins.core.distribution import Distribution
from synthcity.plugins.core.plugin import Plugin
from synthcity.plugins.core.schema import Schema


class DummyGeneratorPlugin(Plugin):
    """Dummy plugin for debugging.
    """

    def __init__(self, **kwargs: Any) -> None:
        super().__init__(**kwargs)

    @staticmethod
    def name() -> str:
        return "dummy_generator"

    @staticmethod
    def type() -> str:
        return "debug"

    @staticmethod
    def hyperparameter_space(*args: Any, **kwargs: Any) -> List[Distribution]:
        return []

    def _fit(self, X: pd.DataFrame, *args: Any, **kwargs: Any) -> "ZeroOnePlugin":
        return self

    def _generate(self, count: int, syn_schema: Schema, **kwargs: Any):
        result = self.schema().sample(count) 
        result[syn_schema.features()] = syn_schema.sample(count)
        
        return result

In [9]:
generators.add("dummy_generator", DummyGeneratorPlugin)

generators.list()

['rtvae',
 'copulagan',
 'pategan',
 'tvae',
 'gaussian_copula',
 'adsgan',
 'marginal_distributions',
 'bayesian_network',
 'uniform_sampler',
 'privbayes',
 'nflow',
 'ctgan']

In [10]:
# Train the new plugin

gen = generators.get("dummy_generator")

gen.fit(X)

In [11]:
# Generate some new data

gen.generate(count = 10)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,20.829528,10.678843,82.912610,1041.686468,0.117809,0.303552,0.116128,0.070000,0.134491,0.082270,...,19.379584,24.050155,215.878013,1668.902380,0.140049,0.490810,0.140529,0.011152,0.650492,0.109360
1,25.328081,21.758706,86.644290,2209.350256,0.160685,0.193565,0.241263,0.136751,0.267606,0.068464,...,36.024670,16.909244,68.841019,2109.624846,0.083718,0.864576,0.829222,0.003406,0.520567,0.104556
2,24.729964,38.193635,115.755923,256.605511,0.073086,0.221167,0.202909,0.108845,0.206223,0.081758,...,14.762381,43.731875,123.607467,1991.068731,0.133708,0.918311,0.201942,0.029372,0.490268,0.080027
3,7.872865,21.815948,175.948462,1665.624338,0.052853,0.149141,0.334780,0.101975,0.265457,0.089124,...,13.766047,21.592989,120.829646,1318.822100,0.107506,0.981591,0.624102,0.114264,0.446505,0.086206
4,17.589181,22.461519,136.420397,1133.375259,0.107622,0.154641,0.146142,0.125727,0.302873,0.073850,...,25.431457,47.283357,137.379979,348.030026,0.094914,0.331210,0.060833,0.092938,0.485911,0.174303
5,21.584569,14.002945,170.327789,467.425719,0.125943,0.078932,0.012561,0.022610,0.116332,0.054352,...,19.746777,25.309541,216.617546,2358.210370,0.142999,0.224414,0.827168,0.254464,0.516923,0.178617
6,18.132497,34.440386,129.483853,184.025511,0.126425,0.285749,0.052338,0.059991,0.113122,0.061187,...,14.289531,45.313974,113.339068,1237.414508,0.080529,0.710828,0.013405,0.249665,0.458899,0.171288
7,26.803717,15.321132,117.571535,287.257154,0.123493,0.224733,0.416186,0.080738,0.284427,0.065832,...,23.404402,43.321080,166.208795,1705.107911,0.220839,0.628189,0.138710,0.181419,0.361167,0.055465
8,27.946201,13.020511,167.883746,819.177644,0.089879,0.288486,0.126724,0.137480,0.271373,0.065039,...,16.515198,23.984920,88.998754,2538.428168,0.166634,0.650063,0.844255,0.087447,0.453815,0.092522
9,10.089519,13.567476,71.043621,2023.113625,0.110852,0.071222,0.341445,0.195865,0.284599,0.082379,...,33.555184,22.398741,162.656020,2284.226492,0.145840,0.127959,0.065477,0.071401,0.452947,0.201213


In [12]:
# Custom generation constraints

from synthcity.plugins.core.constraints import Constraints

constraints = Constraints(rules = [("worst radius", ">", 15)])

generated = gen.generate(count = 10, constraints = constraints)

assert (generated["worst radius"] > 15).any()

generated

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,20.34,15.79,85.24,704.4,0.10410,0.28390,0.01342,0.03438,0.1861,0.05669,...,4.339766e+08,28.00,83.61,508.1,0.1504,0.15250,0.358200,0.11810,0.3218,0.06589
1,11.06,23.21,85.24,516.4,0.07838,0.12980,0.05699,0.08089,0.1942,0.05821,...,1.061987e+09,34.66,76.91,767.3,0.1434,0.10490,0.125500,0.13570,0.3301,0.06688
2,14.11,13.72,95.54,886.3,0.09444,0.12040,0.04209,0.13220,0.1792,0.05890,...,8.302656e+08,19.16,97.96,440.8,0.1227,0.41160,0.179100,0.05781,0.2829,0.09964
3,11.52,14.23,73.06,710.6,0.10360,0.04751,0.11550,0.04107,0.1514,0.05419,...,1.593480e+09,28.01,62.06,517.8,0.1338,0.17930,0.007977,0.00000,0.2250,0.10160
4,20.60,20.52,82.89,684.5,0.08682,0.18680,0.01462,0.02864,0.1707,0.06758,...,1.971911e+09,29.16,86.82,680.6,0.1166,0.28670,0.224100,0.08263,0.2806,0.08490
5,10.49,22.41,123.70,817.7,0.10660,0.05326,0.01657,0.03275,0.1935,0.05661,...,8.366914e+08,22.46,83.09,605.5,0.1434,0.07974,0.178300,0.09594,0.2362,0.09702
6,12.72,14.23,87.50,422.9,0.07966,0.12060,0.01502,0.12550,0.1467,0.06641,...,9.207356e+08,26.44,81.23,2477.0,0.1427,0.05332,0.315100,0.15950,0.1988,0.13410
7,20.20,16.02,129.50,558.1,0.08974,0.16970,0.09447,0.05303,0.1954,0.06072,...,1.257827e+09,30.73,84.42,549.1,0.1883,0.26100,0.628200,0.08194,0.4863,0.08321
8,16.13,16.02,133.80,512.2,0.08284,0.14380,0.04069,0.04938,0.1695,0.05715,...,1.335708e+09,36.00,152.90,547.8,0.1504,0.25340,0.639900,0.18270,0.3397,0.07918
9,17.68,27.15,89.78,571.0,0.08451,0.06307,0.03800,0.02257,0.1769,0.06915,...,1.808690e+09,26.34,133.50,1748.0,0.1551,0.41260,0.160400,0.02500,0.3672,0.07999
